In [123]:
import json, os
from bs4 import BeautifulSoup
import requests
os.chdir(r'T:\PROJECTS\Market_Risk\Damien\EViews_VScode\dev-resources')

In [112]:
def get_soup(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, features="html.parser")
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()



In [114]:
url = 'https://help.eviews.com/content/mathapp-Workfile___Informational_Functions.html' #Workfile & info functions
soup = get_soup(url)
for div in soup.body.find_all('div', class_="Heading_4"):
    print(f'[{div.text}]: mathapp-Workfile___Informational_Functions.html#{div["id"]}')
for table in soup.body.find_all('table'):
    for tr in table.find_all('tr'):
        for c in tr.children:
            div = c.find('div')
            if div.has_attr('id'):
                key = ' '.join(c.stripped_strings).split('(',1)[0]
                if key!='Function':
                    print(f'{key}: mathapp-Workfile___Informational_Functions.html{div["id"]}')
            break

@env: mathapp-Workfile___Informational_Functions.htmlww147009
@fileexist: mathapp-Workfile___Informational_Functions.htmlww141098
@folderexist: mathapp-Workfile___Informational_Functions.htmlww147016
@tablenames: mathapp-Workfile___Informational_Functions.htmlww141104
@getnextname: mathapp-Workfile___Informational_Functions.htmlww141110
@vernum: mathapp-Workfile___Informational_Functions.htmlww145961
@verstr: mathapp-Workfile___Informational_Functions.htmlww141126
@wdir: mathapp-Workfile___Informational_Functions.htmlww145971
@wquery: mathapp-Workfile___Informational_Functions.htmlww147022
@elem: mathapp-Workfile___Informational_Functions.htmlww141132
@first: mathapp-Workfile___Informational_Functions.htmlww141138
@ifirst: mathapp-Workfile___Informational_Functions.htmlww141144
@ilast: mathapp-Workfile___Informational_Functions.htmlww141150
@last: mathapp-Workfile___Informational_Functions.htmlww141156
@dbname: mathapp-Workfile___Informational_Functions.htmlww158704
@ispanel: mathapp-W

In [92]:
url = 'https://help.eviews.com/content/progref-Programming_Language_Reference.html' #Programming ref
soup = get_soup(url)
for link in soup.body.find_all('a'):
    if link.has_attr('title'):
        print(f'{link["title"]}: {link["href"].replace("../content/","")}')

Twitter: #
Facebook: #
LinkedIn: #
Google+1: #
Print: #
E-Mail: #
EViews Programming: cprogram-EViews_Programming.html#ww113
call: progref-call.html#ww738456
else: progref-else.html#ww743132
endif: progref-endif.html#ww738527
endsub: progref-endsub.html#ww738561
exitloop: progref-exitloop.html#ww738631
for: progref-for.html#ww870469
if: progref-if.html#ww738706
include: progref-include.html#ww738739
next: progref-next.html#ww738810
return: progref-return.html#ww738972
sleep: progref-sleep.html#ww883846
step: progref-step.html#ww739071
stop: progref-stop.html#ww739106
subroutine: progref-subroutine.html#ww739141
then: progref-then.html#ww805036
to: progref-to.html#ww739216
wend: progref-wend.html#ww852316
while: progref-while.html#ww739308
addin: progref-addin.html#ww869563
clearerrs: progref-clearerrs.html#ww868115
commandcap: commandcmd-commandcap.html#ww2523090
exec: progref-exec.html#ww869125
logclear: commandcmd-logclear.html#ww2639568
logmode: commandcmd-logmode.html#ww2639698
log

In [129]:
import re
#Commands

'''Object page summary (like Equation, Series etc)
    Object_Section -> short_description
    Object_Description -> skip
    Object_Reference -> usage
then grab all divs until another Object_Section is encountered -> long_description

'''

'''Short listing of functions in table (e.g., Basic Math Functions: https://www.eviews.com/help/helpintro.html#page/content%2Fmathapp-Basic_Mathematical_Functions.html%23)
class=Function_3Col
    class=div id="refid" -> get parent table row -> extract all columns col1 = func+args, col2=short description, col3=long description
'''

'''Command entry structure (Applies to a wide variety of commands and functions including object methods) -- can also be multipage that starts with a certain tag ID
class=Command_Description --> [short description]
class=Command_Section, text contains "Syntax" --> drop but marks the start of syntax block
class=Program_Syntax|Command_Type.*
    class Command_Summary --> text contains [command name]
    class Argumnet -> text contains [argument(s)]
class=Command_Type.*
    argument explanations including return types e.g., "Returns string"
class=Body_Text --> [long description], including of arguments stitch together multiple blocks of Body_Text
    ...
class=Command_Section, text contains "Options" --> skip but marks the start of options table/description
class=Options_Table -> [options description] by looping over rows and cols.
class=Command_Section, text contains "Examples" --> drop
class=Command_Section, text contains "Cross-References" --> drop
class=Command_Description --> Next command in multicommand page, don't go past this point
'''

keywords = json.load(open(r"keywords.json"))
pages = json.load(open(r"docpacket.json"))

def strip_image(tag):
    img = res.find('img', class_="Default EquationGraphic")
    while img!=None:
        txt = img["src"]
        img.replace_with(f'[img:{txt}]')
        img = res.find('img', class_="Default EquationGraphic")

def get_text(tag):
    return ' '.join([c for c in tag.stripped_strings])

for grp in keywords:
    for kw in keywords[grp]:
        kwinfo = keywords[grp][kw]
        res = BeautifulSoup(pages[kwinfo['uri']]).body
        strip_image(res)
        if kwinfo['anchor']!='':
            res = res.find(id=kwinfo['anchor'])
            if res==None:
                print('*', grp, kw, 'ERROR - ANCHOR NOT FOUND', kwinfo)
                continue
            parent = res.find_parent('td')
            if parent!=None: #DOCTYPE A1: this looks like function docs on one row of a 3 col table
                res = parent
                attribs = ['usage','description','long_description']
                table = res.find_parent('table')
                if table!=None and 'Function_3col_2desc' in table['class']:
                    step = res.find_previous_sibling()
                    if step!=None:
                        res = step
                    attribs = ['description','usage','long_description']
                if table!=None and 'Mathapp' in table['class']:
                    attribs = ['usage','description']
                for a in attribs:
                    kwinfo[a] = get_text(res)
                    res = res.find_next_sibling()
                    if res==None: break
                print('*', grp, kw, *[f'{a}={kwinfo[a]}' for a in kwinfo])
                continue
            if res.has_attr("class") and "Object_Reference" in res['class']: #DOCTYPE A2: this looks like short object property docs in a div ##TODO: Seek back to the class_="Object_Subsection" and get the text with the type
                cs = res.find(class_="Command_Summary")
                if cs!=None:
                    text = get_text(res)
                    m = re.match(rf'^({kw}(\(.*?\))?)\s*(.*)', text, re.IGNORECASE)
                    if m!=None:
                        kwinfo['usage'] = m.group(1)
                        kwinfo['description'] = m.group(3)
                    else:
                        print('*', grp, kw, 'BAD MATCH:', text)
                        continue
                print('*', grp, kw, *[f'{a}={kwinfo[a]}' for a in kwinfo])
                continue

        res0 = res
        res = res.find(class_=["Command_Title_Box","Object_Section","Object_Description"])
        if res == None:
            print('*', grp, kw, 'ERROR - NO MATCHING PATTERN', kwinfo)
        elif "Command_Title_Box" in res["class"]: #DOCTYPE B: this looks like a command description doc
            res = res.find_next_sibling() #Div with Table data for command_title_box -- skip it
            long_doc = []
            while res!=None:
                res = res.find_next_sibling()
                if res==None or res.name=='footer':
                    break
                elif res.has_attr("class") and ("Command_Section" in res['class'] or "Object_Section" in res['class']):
                    if res==None or get_text(res).strip().lower()!='syntax': break
                    #skip this entry otherwise
                elif res.has_attr("class") and ("Program_Syntax" in res['class'] or "Command_Type" in res['class'] or 
                                                "Command_Types" in res['class'] or "Command_Types_Tabbed" in res['class']): 
                     kwinfo['usage'] = kwinfo['usage']+'\n'+get_text(res) if 'usage' in kwinfo else get_text(res)
                else: long_doc.append(get_text(res))
            if len(long_doc)>0:
                kwinfo['description'] = long_doc[0]
            if len(long_doc)>1:
                kwinfo['long_description'] = '\n'.join(long_doc[1:])
            if res!=None and res.text.lower().strip()!='options':
                options = {} ##TODO: There can be more than one table.
                res = res.find('table', class_="Options_Table")
                if res!=None:
                    res = res.find('tr')            
                    while res!=None:
                        opt = get_text(res.children[0])
                        text = get_text(res.children[1])
                        options[opt] = text
                        res = res.find_next_sibling()
                    if(len(kwinfo)>0):
                        kwinfo['options'] = options
            print('*', grp, kw, *[f'{a}={kwinfo[a]}' for a in kwinfo])
        elif "Object_Section" in res["class"] or "Object_Description" in res["class"]: #DOCTYPE C: this looks like Object documentation
            kwinfo['description'] = get_text(res)
            res = res.find(class_='Object_Reference')
            long_doc = []
            while res!=None:
                res = res.find_next_sibling() #Note that we skip the first line of text because it is just a link to the usage documenation
                if res==None or res.has_attr("class") and "Command_Section" in res["class"]: break
                long_doc.append(get_text(res))
            if len(long_doc)>0:
                kwinfo['long_description'] = '\n'.join(long_doc)
            print('*', grp, kw, *[f'{a}={kwinfo[a]}' for a in kwinfo])
json.dump(keywords, open("keywords_detail.json", "w"))
json.dump(keywords, open("../resources/keywords_detail.json", "w"))

* Equation [Equation] uri=equationcmd-Equation anchor= description=Equation object. Equations are used for single equation estimation, testing, and forecasting.
* Equation @aic uri=equationcmd-Equation anchor=ww325952 usage=@aic description=Akaike information criterion.
* Equation @ardlceccoefs uri=equationcmd-Equation anchor=ww461539 usage=@ardlceccoefs description=returns a vector of coefficient estimates from the conditional error-correction (CEC) regression in univariate (N)ARDL estimation.
* Equation @ardlcoint BAD MATCH: @ardlceccoefs returns a vector of coefficient estimates from the conditional error-correction (CEC) regression in univariate (N)ARDL estimation.
* Equation @ardlcointsubst uri=equationcmd-Equation anchor=ww360177 usage=@ardlcointsubst description=returns string representation of the cointegration form of an ARDL equation with substituted coefficients.
* Equation @ardleccoefs uri=equationcmd-Equation anchor=ww623108 usage=@ardleccoefs description=returns a vector 